# Import data and drop irrelevant columns

In [11]:
import lazypredict
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

#Read data file
import pandas as pd

filepath = "trial_298.csv"
df = pd.read_csv(filepath)
features = df

# Remove the labels from the features
features = features.drop('protection_level', axis = 1)
features = features.drop('page_title', axis = 1)
features = features.drop('protection_expiry', axis = 1)
features = features.drop('page_id', axis = 1)
features = features.drop('page_freshness', axis = 1)

features

,view_count,page_length,number_page_watchers,number_page_watchers_recent_edits,number_of_redirects,page_views_past_30days,total_edits,recent_number_of_edits,number_distinct_authors,number_categories
0,4433720.00,22427,76,9,4,80115,864.00,1.00,1.00,20.00
1,3648535.00,24897,81,12,0,55019,962.00,8.00,4.00,26.00
2,3155183.00,33119,244,20,14,46350,946.00,7.00,6.00,15.00
3,3151197.00,4265,34,8,0,26392,60.00,0.00,0.00,6.00
4,2885266.00,18081,Fewer than 30 watchers,NaN,0,10757,193.00,0.00,0.00,15.00
...,...,...,...,...,...,...,...,...,...,...
1199,432.00,32,Fewer than 30 watchers,NaN,0,0,2.00,0.00,0.00,nan
1200,852.00,32,Fewer than 30 watchers,NaN,0,0,4.00,0.00,0.00,nan
1201,432.00,32,Fewer than 30 watchers,NaN,0,0,2.00,0.00,0.00,nan
1202,41692.00,49,Fewer than 30 watchers,NaN,0,60,3.00,0.00,0.00,nan


## Data wrangling

In [12]:
# Labels are the values we want to predict
labels = np.array(df['protection_level'])

# 0 => unprotected
# 1 => autoconfirmed
# 2 => extendedconfirmed
# 3 => sysop
labels_encoded = []
for item in labels:
    if(item =="unprotected"):
        labels_encoded.append(0)
    elif(item == "autoconfirmed"):
        labels_encoded.append(1)
    elif(item == "extendedconfirmed"):
        labels_encoded.append(2)
    elif(item == "sysop"):
        labels_encoded.append(3)  

#np.nan convers
features = features.replace('Fewer than 30 watchers',np.NaN)
features = features.replace('There may or may not be a watching user visiting recent edits',np.NaN)

features['page_length'] = features['page_length'].astype(float)
features['total_edits'] = features['total_edits'].astype(float)
features['number_page_watchers'] = features['number_page_watchers'].astype(float)
features['number_page_watchers_recent_edits'] = features['number_page_watchers_recent_edits'].astype(float)

# Saving feature names for later use
feature_list = list(features.columns)

features

,view_count,page_length,number_page_watchers,number_page_watchers_recent_edits,number_of_redirects,page_views_past_30days,total_edits,recent_number_of_edits,number_distinct_authors,number_categories
0,4433720.00,22427.00,76.00,9.00,4,80115,864.00,1.00,1.00,20.00
1,3648535.00,24897.00,81.00,12.00,0,55019,962.00,8.00,4.00,26.00
2,3155183.00,33119.00,244.00,20.00,14,46350,946.00,7.00,6.00,15.00
3,3151197.00,4265.00,34.00,8.00,0,26392,60.00,0.00,0.00,6.00
4,2885266.00,18081.00,nan,nan,0,10757,193.00,0.00,0.00,15.00
...,...,...,...,...,...,...,...,...,...,...
1199,432.00,32.00,nan,nan,0,0,2.00,0.00,0.00,nan
1200,852.00,32.00,nan,nan,0,0,4.00,0.00,0.00,nan
1201,432.00,32.00,nan,nan,0,0,2.00,0.00,0.00,nan
1202,41692.00,49.00,nan,nan,0,60,3.00,0.00,0.00,nan


In [13]:
#Apply KNN Imputer for missing values
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
features = imputer.fit_transform(features)


from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels_encoded, test_size =0.20, random_state = 53)

X_train = train_features
y_train = train_labels
X_test = test_features
y_test = test_labels


from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 16.38it/s]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
LGBMClassifier                     0.77               0.77    None      0.76   
RandomForestClassifier             0.75               0.75    None      0.74   
XGBClassifier                      0.74               0.75    None      0.74   
ExtraTreesClassifier               0.73               0.73    None      0.72   
BaggingClassifier                  0.70               0.70    None      0.69   
DecisionTreeClassifier             0.67               0.67    None      0.67   
KNeighborsClassifier               0.62               0.62    None      0.60   
NuSVC                              0.61               0.62    None      0.60   
LabelSpreading                     0.61               0.61    None      0.58   
LinearSVC                          0.60               0.61    None      0.56   
LogisticRegression                 0.60 

# Best Model


In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score
import lightgbm as lgb

#Instantiate model
model = lgb.LGBMClassifier()

# fit model no training data
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

print("Model: LGBM")

# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.4f%%" % (accuracy * 100.0))

balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy: %.4f%%" % (balanced_accuracy * 100.0))

f1_score = f1_score(y_test, y_pred, average='macro')
print("F1 Score : %.4f%%" % (f1_score * 100.0))

Model: LGBM
Accuracy: 76.3485%
Balanced Accuracy: 76.6068%
F1 Score : 75.8816%


Model: LGBM
Accuracy: 76.3485%
Balanced Accuracy: 76.6068%


TypeError: 'numpy.float64' object is not callable